# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman (*RSA*) cryptosystem demonstrated here combines secure key&ndash;exchange with digital signature to provide for the confidentiality, integrity and authentication of a private message exchanged between two parties (Alice and Bob) over an insecure channel. Moreover, an adversary (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of corrupting it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality, and utilizes a *key&ndash;encapsulation mechanism* (or *KEM*). This differs from the more traditional approach to key&ndash;exchange, based on the *discrete logarithm problem*, introduced by Diffie and Hellman in their 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf) (see the notebooks [*DH.ipynb*](https://nbviewer.org/github/dchampion/crypto/blob/master/doc/DH.ipynb) and [*EC.ipynb*](https://nbviewer.org/github/dchampion/crypto/blob/master/doc/EC.ipynb) for demonstrations of classic Diffie&ndash;Hellman (DH) and *Elliptic&ndash;Curve* Diffie&ndash;Hellman (ECDH), respectively).

Note that the *KEM* presented here lacks a property the Diffie&ndash;Hellman implementations do not, and which some might consider a disqualifying deficiency; this property is known as [*perfect forward secrecy*](https://en.wikipedia.org/wiki/Forward_secrecy). To overcome this deficiency, the digital signature component of this implementation may nevertheless be used together with Diffie&ndash;Hellman style key agreement (DH or ECDH) to implement a robust cryptosystem. The *KEM* implementation is demonstrated here only to illustrate its use.

The digital signature component of this cryptosystem provides for message integrity and authentication, ideas that were conceived by Diffie and Hellman in their 1976 paper, but not implemented until Rivest, Shamir and Adleman published their 1978 paper [*A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems*](https://people.csail.mit.edu/rivest/Rsapaper.pdf).

## The Source Code
The following sequence of function calls to the `rsa` module (located in the [src](https://github.com/dchampion/crypto/tree/master/src) folder of this repository) illustrates a session in which Alice signs and encrypts a private message, and then sends the message to Bob over an insecure channel. Meanwhile, an adversary (Eve) intercepts the message.

It is assumed that Alice, Bob and Eve all possess their own copy of the `rsa` module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must *not* be transmitted over the insecure channel).

Explanations of behavior appear *below* the code snippets and their outputs.

In [1]:
import path_resolver
import rsa
import sym

Import the `rsa` module, and a module `sym` that implements an elementary symmetric cipher that Alice and Bob will use in the protocol.

In [2]:
pA, qA, nA, d3A, d5A = rsa.generate_rsa_key(2048)

The sequence begins with Alice generating her RSA key parameters. She does this by calling the `rsa.generate_rsa_key` function, passing it a modulus size of `2048` bits. The function returns the quintuple *pA*, *qA*, *[nA]*, *d3A* and *d5A* (the suffix *A* on all the parameters identify them as belonging to Alice). Alice transmits her public modulus *[nA]* to Bob.

Alice must keep the values *pA* and *qA*&mdash;the 1024&ndash;bit prime factors of *[nA]*&mdash;secret. She must also keep *d3A* and *d5A*&mdash;her signing and decryption keys, respectively&mdash;secret.

To get an idea of the size and proportions of these parameters, let's print some of them out.

In [3]:
print(pA)

94254439759509774264198559926893556951645824202291769199568286336373456590722418452935271067654692839329548374494514929899018533438873894056039967226452237569280901080385755965356992494587956205391494946645680744478831175703750291597372173573490085724932895519632998439444801242184138981041565832450710810727


In [4]:
print(qA)

156473806473406808649235113082354772890230806165637122109529285193550150319123998983709851594835497644623266136210886476829779045603761087636273866947342921273963826902977869158349776591039775614434359181949774742762676613079420292287353556260708350148900096405755253662501355042746605857295041477349988062433


In [5]:
print(nA)

14748350966188912600891099946474229645600984320856784039154217325292234808314898066216951413771277305054549409380049156390827440599208856114762253451255273466746997954575775654885134818292086753778015287875566600490932455144782222496574253678668343961593660223558050539706110818778139051668565028965447381184095769916871236392495951952148616290751684201743169506766602260475256309856386229657911657054724668445840745782493996268092926180056258086067911962281392975998550951650136130098422486158370805699976198648787062033646401523788081497314222576833747012761240041698133462621267459214113421271600516398949322118791


 Note that *[nA]*, which is the product of *pA* and *qA*, is twice the length of each individually; *pA*'s and *qA*'s ~310 decimal digits to *[nA]*'s ~620 decimal digits.

Recall that Alice requested a modulus of 2048 bits in length in her call to `rsa.generate_rsa_key`. Next, let's print the bit lengths of *[nA]* and its prime factors.

In [6]:
print(nA.bit_length())

2047


In [7]:
print(pA.bit_length())

1024


In [8]:
print(qA.bit_length())

1024


What we see is that a decimal number of ~620 digits equates to a bit&ndash;length of 2048 (or reasonably close to it), and further that a decimal number of ~310 digits is 1024 bits long.

Next, let's check to see if *[nA]* is indeed the product of *pA* and *qA* (note this step is not necessary for the protocol; it is done here just to prove *[nA]* is in fact the product of *qA* and *pA*).

In [9]:
print(qA * pA == nA)

True


Now it's Bob's turn to generate RSA keys of his own. To differentiate Bob's keys from Alice's keys, we append Bob's with the captial letter *B*.

In [10]:
pB, qB, nB, d3B, d5B = rsa.generate_rsa_key(2048)

Because Bob requested a modulus size of 2048, based on our inspection of Alice's keys above we can conclude that Bob's keys are the same size as Alice's keys; so we don't need to print their values here.

Now that Alice and Bob have generated their keys, and transmitted to each other their public moduli *[nA]* and *[nB]*, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob.

In [11]:
mA = "Sign and encrypt me!"

First, Alice stores the message "Sign and encrypt me!" in the variable `mA`.

In [12]:
oA = rsa.sign(d3A, pA, qA, mA)

Then Alice calls the `rsa.sign` fuction, passing it her secret signing key *d3A*, the two factors of *[nA]*, *pA* and *qA*, and the message *mA*, to sign the message.

She stores the signature in the variable *[oA]*.

In [13]:
KA, cA = rsa.encrypt_random_key(nB, rsa.ENCRYPTION_EXPONENT)

Next, Alice calls the `rsa.encrypt_random_key` function, passing it Bob's RSA modulus *[nB]* and the global encryption exponent *[rsa.ENCRYPTION_EXPONENT]*. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., 3DES, AES). The function returns the pair *KA* and *[cA]*, which are, respectively, the symmetric encryption key, and the ciphertext of its input material; the same material Bob will use to replicate the symmetric key on his side of the channel.

Alice must keep the session key *KA* private, but she transmits the ciphertext *[cA]* to Bob.

In [14]:
mAC = sym.encrypt(KA, mA)

Next, Alice encrypts the private message, calling the symmetric cipher function `sym.encrypt` and passing it her symmetric key *KA* and the plaintext of her message *mA* (note that in a real application, Alice and Bob would use an industrial&ndash;strength cipher (e.g., 3DES or AES) for message encryption).

The result of this operation is the ciphertext *[mAC]*.

In [15]:
print(mAC)

105941919010361689034877669951054732672941445807703844975621392204994534434875


Here we print the ciphertext to demonstrate that, if Eve were to intercept it, she would see nothing but a seemingly random string of numbers.

Next, Alice transmits the message signature *[oA]*, the message ciphertext *[mAC]* and the ciphertext of the symmetric&ndash;key's input material *[cA]* to Bob.

In [16]:
KB = rsa.decrypt_random_key(d5B, cA, pB, qB)

In order to decrypt the message, Bob must recover the symmetric key Alice used to encrypt it. He does this by calling  `rsa.decrypt_random_key`, passing it his private decryption key *d5B*, the ciphertext of Alice's symmetric&ndash;key input material *[cA]*, and the two prime factors of his public modulus *pB* and *qB*.

In [17]:
print(KB == KA)

True


Here we prove that the key Bob decrypted above, *KB*, is the same as the key Alice encrypted, *KA* (again, this step is not required for the protocol; it is only used here to prove the keys are equal).

In [18]:
mB = sym.decrypt(KB, mAC)

Bob calls `sym.decrypt`, passing it *KB* and the ciphertext of Alice's message *[mAC]*, to recover Alice's plaintext message *mA*.

Bob stores the result in *mB*.

In [19]:
print(mB == mA)

True


Here we prove that the recovered plaintext indeed matches Alice's original message.

In [20]:
print(mB)

Sign and encrypt me!


Let's print the message, just to make sure.

In order to complete the session, all that remains is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

In [21]:
print(rsa.verify(nA, rsa.VERIFICATION_EXPONENT, mB, oA))

True


Bob calls the `rsa.verify` fuction, passing Alice's public modulus *[nA]*, the constant signature&ndash;verification key *[rsa.VERIFICATION_EXPONENT]*, the decrypted message *mB* and the sigature sent to him by Alice *[oA]*.

If the assertion above holds, Bob can be assured that the message "Sign and encrypt me!", even if intercepted by Eve, is known only to him and Alice, and that the message was not corrupted in any way by Eve.